In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from netCDF4 import Dataset
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde
from sklearn.metrics import r2_score
from scipy.stats import pearsonr
from PIL import Image
import seaborn as sns
cm = plt.get_cmap('jet') 
import pickle
from sklearn.ensemble import RandomForestRegressor

In [4]:
# observation
obs_LOSS_mean = np.flipud(np.array(Image.open('ML_LOSS_mean_50degree_constrain.tif')))
obs_LOSS_SD = np.flipud(np.array(Image.open('ML_LOSS_SD_50degree_constrain.tif')))
obs_LOSS_mean[obs_LOSS_mean<0]= np.nan
obs_LOSS_SD[obs_LOSS_SD<0] = np.nan

pre = np.flipud(np.array(Image.open('Pre_his_con_05.tif')))
temp = np.flipud(np.array(Image.open('Tem_his_con_05.tif')))
pre[pre>1e10]= np.nan
temp[temp>1e10]= np.nan

# model 
LOSS = np.flipud(np.array(Image.open('SEIB_biomassloss_con_05.tif')))
NPP = np.flipud(np.array(Image.open('NPP_Grid_SEIB_con_05_sum_final.tif')))
LOSS[LOSS<0]= np.nan
NPP[NPP<0]= np.nan

print(NPP)



[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]


In [ ]:
np.random.seed = 0
n_ensemble = 100      # this used 100 bootstrapping

rf_y_corrected_uq_ensemble = np.full([6734,n_ensemble],np.nan)

for uq_ensembles in range(n_ensemble):
    alldata = []
    for i in range(720):
        for j in range(279):
            
            obs_LOSS_ensemble = obs_LOSS_mean[j,i] + np.random.normal(loc=0.0, scale=np.nanmean(obs_LOSS_SD[j,i]))
            
            if ~np.isnan(obs_LOSS_ensemble) and ~np.isnan(LOSS[j,i]) and \
            ~np.isnan(temp[j,i]) and ~np.isnan(pre[j,i]) and \
            ~np.isnan(NPP[j,i]):
                alldata.append(np.hstack((j,i,temp[j,i],pre[j,i],LOSS[j,i],NPP[j,i],obs_LOSS_ensemble)))
    alldata= np.array(alldata)
    X = alldata[:,[2,3,4]]        # this accounts for climate: temperature and precipitation
    
    y = alldata[:,5]
    
    X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=0)
    
    # build RF
    rf = RandomForestRegressor(n_estimators = 1000, max_depth = 15, max_features = 'sqrt', criterion='mse', random_state=0)
    
    rf.fit(X_train,y_train)
    '''
    pkl_filename = "rf.pkl"
    with open(pkl_filename, 'wb') as file:
        pickle.dump(rf, file)
    
    pkl_filename = "rf.pkl"
    with open(pkl_filename, 'rb') as file:
        rf = pickle.load(file)
    '''   
    #joblib.save(rf, "RF.joblib")
    #rf = joblib.load("RF.joblib")
    
    #y_pred_train = rf.predict(X_train)
    #y_pred_test = rf.predict(X_test)
    #rf_y =  np.vstack((y_pred_train.reshape(-1,1),y_pred_test.reshape(-1,1)))
    #data_y  = np.vstack((y_train.reshape(-1,1),y_test.reshape(-1,1)))

    rf_y_corrected  = rf.predict(alldata[:,[2,3,6]])
    rf_y_corrected_uq_ensemble[:,uq_ensembles] = rf_y_corrected
    
    



<ipython-input-5-6d545ee06033>:11: RuntimeWarning: Mean of empty slice
  obs_LOSS_ensemble = obs_LOSS_mean[j,i] + np.random.normal(loc=0.0, scale=np.nanmean(obs_LOSS_SD[j,i]))
<ipython-input-5-6d545ee06033>:11: RuntimeWarning: Mean of empty slice
  obs_LOSS_ensemble = obs_LOSS_mean[j,i] + np.random.normal(loc=0.0, scale=np.nanmean(obs_LOSS_SD[j,i]))
<ipython-input-5-6d545ee06033>:11: RuntimeWarning: Mean of empty slice
  obs_LOSS_ensemble = obs_LOSS_mean[j,i] + np.random.normal(loc=0.0, scale=np.nanmean(obs_LOSS_SD[j,i]))
<ipython-input-5-6d545ee06033>:11: RuntimeWarning: Mean of empty slice
  obs_LOSS_ensemble = obs_LOSS_mean[j,i] + np.random.normal(loc=0.0, scale=np.nanmean(obs_LOSS_SD[j,i]))
<ipython-input-5-6d545ee06033>:11: RuntimeWarning: Mean of empty slice
  obs_LOSS_ensemble = obs_LOSS_mean[j,i] + np.random.normal(loc=0.0, scale=np.nanmean(obs_LOSS_SD[j,i]))
<ipython-input-5-6d545ee06033>:11: RuntimeWarning: Mean of empty slice
  obs_LOSS_ensemble = obs_LOSS_mean[j,i] + np.ran

In [8]:

# convert to 3D array
ML_corrected_3D = np.full([720,279,n_ensemble],np.nan)
for i in range(6734):
    ML_corrected_3D[int(alldata[i,1]),int(alldata[i,0]),:] = rf_y_corrected_uq_ensemble[i,:]

# output to 3D array
np.save('ML_corrected_climate_grid_NPP_SEIB.npy', ML_corrected_3D)

